### Option Strategy Evaluation (Module 1)
Calculate returns across all permutations for various option strategies using the SPY BigQuery Dataset. The results can then be compared using a Data Visualization Tool to determine the optimal place at which to position strike prices. Returns will be calculated for the following strategies.
- Buy Call
- Buy Put
- Buy Call Spread
- Buy Put Spread
- Write Call Spread
- Write Put Spread
- Buy Iron Condor
- Write Iron Condor


The holding period for all calculations will be 1 week and the expiry date will vary from 1 to 5 weeks out. This will enable us to determine whether holding options through to expiry is the best strategy or if there is some optimal holding period relative to the time remaining on an option.

The above option strategies fall into 1 of 3 categories; Bullish, Bearish and Neutral. Choosing a strategy therefore implies an expectation as to the future behavior of the underlying stock. In order to evaluate trades in the context of this expectation, we need to identify the direction in which the market moved for each week in the study. The market_direction field will be derived as follows using the At-The-Money Implied Volatility.
- Strong Up - up more than the average expected move (upper 25% of a Normal Distribution)
- Neutral Up - up less than the average expected move (between 0% and upper 25% of a Normal Distribution)
- Neutral Down - down less than the average expected move (between 0% and lower 25% of a Normal Distribution)
- Strong Down - down more than the average expected move (lower 25% of a Normal Distribution)

The BigQuery SPY table has a sampling_key field (random number between 0 and 1) associated with each quote record which will help us determine the various returns that can be expected based on the accuracy of a model. To illustrate, consider a binary model that predicts SPY moving either up or down with a 70% accuracy rate. We could simulate this model by having it make correct predictions 70% of the time. In other words, we would have our simulated model predict that SPY will move up if it did in fact move up and the quote record had a sampling_key greater than .3. Likewise our model would predict that SPY will move down if it did in fact move down and the quote record had a sampling_key greater than .3 (note: 70% of random numbers betweeen 0 and 1 will have a value greater than .3). Using this approach we can simulate the expected returns for arbitrary models with various prediction acurracy rates.

### This Module will create the -raw input files needed to evaluate option strategy returns
- spy_1wk_options
- spy_2wk_options
- spy_3wk_options
- spy_4wk_options
- spy_5wk_options

### Strategy Evaluation Data Elements
- entry_date
- exit_date
- expiry_date
- strike_price
- entry_stock_price
- entry_atm_price
- entry_atm_iv
- entry_call_bid
- entry_call_ask
- entry_call_moneyness
- entry_put_bid
- entry_put_ask
- entry_put_moneyness
- exit_call_bid
- exit_call_ask
- exit_put_bid
- exit_put_ask
- market_direction
- sample_key

In [23]:
%%bigquery bq_results
-- weekly option entry and exit stats for 1 week holding periods
with trade_entry as
(
select quote_week as entry_week, quote_date as entry_date, expiry_date,  strike_price, 
  call_bid as call_entry_bid, call_ask as call_entry_ask, call_moneyness as call_entry_moneyness, 
  put_bid as put_entry_bid, put_ask as put_entry_ask, put_moneyness as put_entry_moneyness, 
  underlying_price as entry_stock_price, atm_price as entry_atm_price, atm_iv as entry_atm_iv, sampling_key as entry_sampling_key
from expiry-week.option_quotes.TEST, UNNEST(expiry_dates), UNNEST(strike_prices)
)
select t1.*, t2.quote_week as exit_week, t2.underlying_price as exit_stock_price, t2s2.call_bid as exit_bid, t2s2.call_ask as exit_ask
from trade_entry t1
inner join expiry-week.option_quotes.TEST t2
  on t1.entry_week + 1 = t2.quote_week
inner join UNNEST(expiry_dates) t2s1
  on t1.expiry_date = t2s1.expiry_date
inner join UNNEST(strike_prices) t2s2
  on t1.strike_price = t2s2.strike_price
order by t1.entry_week


In [24]:
bq_results.head()

,entry_week,entry_date,expiry_date,strike_price,call_entry_bid,call_entry_ask,call_entry_moneyness,put_entry_bid,put_entry_ask,put_entry_moneyness,entry_stock_price,entry_atm_price,entry_atm_iv,entry_sampling_key,exit_week,exit_stock_price,exit_bid,exit_ask
0,265,2015-07-02,2015-07-17,199.500000000,8.380000000,9.140000000,0.879000000,0.840000000,0.860000000,0.121000000,207.320000000,207.400000000,0.164000000,0.837770000,266,207.480000000,8.160000000,8.290000000
1,265,2015-07-02,2015-07-17,200.000000000,8.170000000,8.540000000,0.863000000,0.910000000,0.920000000,0.137000000,207.320000000,207.400000000,0.164000000,0.837770000,266,207.480000000,7.700000000,7.820000000
2,265,2015-07-02,2015-07-17,200.500000000,7.480000000,8.110000000,0.845000000,0.980000000,1.010000000,0.155000000,207.320000000,207.400000000,0.164000000,0.837770000,266,207.480000000,7.250000000,7.370000000
3,265,2015-07-02,2015-07-17,201.000000000,7.260000000,7.690000000,0.827000000,1.060000000,1.100000000,0.173000000,207.320000000,207.400000000,0.164000000,0.837770000,266,207.480000000,6.800000000,6.920000000
4,265,2015-07-02,2015-07-17,201.500000000,6.670000000,7.270000000,0.807000000,1.150000000,1.180000000,0.193000000,207.320000000,207.400000000,0.164000000,0.837770000,266,207.480000000,6.350000000,6.480000000


In [20]:
expiry_dates = df_results['expiry_date'].unique()
sorted_expiry_dates = sorted(expiry_dates)
#for 1 week options the options will have already expired on the exit date so we need to use 
#the exit price of the underlying to evaluate the option returns
#on a similar note we can load all the entry dates and then loag these by n weeks to map to number of weeks to expiry



                                

In [19]:
sorted_expiry_dates

[datetime.date(2015, 7, 17),
 datetime.date(2015, 7, 24),
 datetime.date(2015, 7, 31),
 datetime.date(2015, 8, 7),
 datetime.date(2015, 8, 14),
 datetime.date(2015, 8, 21),
 datetime.date(2015, 8, 28),
 datetime.date(2015, 9, 4),
 datetime.date(2015, 9, 11),
 datetime.date(2015, 9, 18),
 datetime.date(2015, 9, 25),
 datetime.date(2015, 9, 30),
 datetime.date(2015, 10, 2),
 datetime.date(2015, 10, 9),
 datetime.date(2015, 10, 16),
 datetime.date(2015, 10, 23),
 datetime.date(2015, 10, 30),
 datetime.date(2015, 11, 6),
 datetime.date(2015, 11, 13),
 datetime.date(2015, 11, 20),
 datetime.date(2015, 11, 27),
 datetime.date(2015, 12, 4),
 datetime.date(2015, 12, 11),
 datetime.date(2015, 12, 19),
 datetime.date(2015, 12, 24),
 datetime.date(2015, 12, 31),
 datetime.date(2016, 1, 8),
 datetime.date(2016, 1, 15),
 datetime.date(2016, 1, 22),
 datetime.date(2016, 1, 29),
 datetime.date(2016, 2, 5),
 datetime.date(2016, 2, 19),
 datetime.date(2016, 3, 18),
 datetime.date(2016, 3, 31),
 datetim